In [1]:
import sqlite3
import pandas as pd
import pymongo
import numpy as np
import matplotlib.pyplot as plt
import requests

In [2]:
BERLIN_LAT = '52.5200'
BERLIN_LONG = '13.4050'
ENV['DARK_SKY_API_KEY']

NameError: name 'ENV' is not defined

In [49]:
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()
d = conn.cursor()
e = conn.cursor()

In [50]:
c.execute("""SELECT Season, Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR
             FROM Matches
             WHERE Season = 2011""")

In [51]:
df = pd.DataFrame(c.fetchall()).copy()
df.columns = [i[0] for i in c.description]
df_head = df.head().copy()

In [52]:
df_head

,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [53]:
df_head.loc[0,'Date']

'2012-03-31'

In [58]:
c.execute("""DROP VIEW IF EXISTS HomeTeamGoals""")
c.execute("""DROP VIEW IF EXISTS AwayTeamGoals""")
c.execute("""CREATE VIEW HomeTeamGoals
             AS 
             SELECT HomeTeam, SUM(FTHG) as TotHomeGoals
             FROM Matches
             WHERE Season = 2011
             GROUP BY HomeTeam""")
d.execute("""CREATE VIEW AwayTeamGoals
             AS
             SELECT AwayTeam, SUM(FTAG) as TotAwayGoals
             FROM Matches
             WHERE Season = 2011
             GROUP BY AwayTeam""")

In [62]:
e.execute("""SELECT *, TotHomeGoals+TotAwayGoals as TotGoals
             FROM HomeTeamGoals htg
             JOIN AwayTeamGoals atg
             ON htg.HomeTeam = atg.AwayTeam""")

In [63]:
goals_df = pd.DataFrame(e.fetchall()).copy()
goals_df.columns = [i[0] for i in e.description]
goals_df.head()

,HomeTeam,TotHomeGoals,AwayTeam,TotAwayGoals,TotGoals
0,Aachen,15,Aachen,15,30
1,Arsenal,39,Arsenal,35,74
2,Aston Villa,20,Aston Villa,17,37
3,Augsburg,20,Augsburg,16,36
4,Bayern Munich,49,Bayern Munich,28,77


In [64]:
goals_df2 = goals_df[['HomeTeam', 'TotGoals']].copy()

In [65]:
goals_df2.head()

,HomeTeam,TotGoals
0,Aachen,30
1,Arsenal,74
2,Aston Villa,37
3,Augsburg,36
4,Bayern Munich,77


In [66]:
goals_df2.shape

(56, 2)